In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read the data
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')

# get a glimpse over the data
titanic.head()

In [ ]:
# reviewing the data, I find out that all the numerical columns 
# are complete, except for the Age column, which is normal, 
# since some people may have sneaked into the boat to immigrate to America
titanic.describe()

In [ ]:
# since Cabin was an object, it didn't appear in the describe method,
# but now we know it has a lot of null values
titanic.info()

In [ ]:
# the Cabin column has 147 different unique values,
# which makes one hot encoding inefficient
titanic['Cabin'].nunique()

In [ ]:
# the Ticket column just tells you the 
# type of ticket, as "STON/02." may show.
# ordinal encoding may be useful here
titanic['Ticket']

In [ ]:
# in the data description of the competition,
# it says there are 3 ports, 
# C = Cherbourg
# Q = Queenstown
# S = Southampton
# this is perfect for one hot encoding
titanic['Embarked']

In [ ]:
# I will drop this column, since it's useless
titanic['PassengerId']

In [ ]:
# I will create a new feature for the amount of family members
# family members = siblings + spouses + parents + children
titanic['FamMembers'] = titanic['SibSp'] + titanic['Parch']

In [ ]:
# splitting the data into training and validation sets
from sklearn.model_selection import train_test_split

y = titanic['Survived']
X = titanic.drop('Survived', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
# I drop PassengerId column
X_train.drop('PassengerId', axis=1, inplace=True)
X_valid.drop('PassengerId', axis=1, inplace=True)

In [ ]:
# separating the data between numerical and categorical columns.
# since the dataset doesn't have too many columns, I will keep all
numerical_cols = X_train.select_dtypes(include=['int64','float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

In [ ]:
# we obtain One Hot Encoding, Ordinal Encoding and Target Encoding columns
ohe_cols = [col for col in categorical_cols if X_train[col].nunique() <= 3]
ordinal_cols = [col for col in categorical_cols if 3 < X_train[col].nunique() < 10]
high_cardinality_cols = [col for col in categorical_cols if X_train[col].nunique() >= 10]

In [ ]:
# fill null values with the median in numerical columns
from sklearn.impute import SimpleImputer

num_cols_transformer = SimpleImputer(strategy='median')

In [ ]:
# pipelines for categorical columns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline

ohe_cols_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore',
                           sparse_output=False))
])

ordinal_cols_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1))
])

high_cardinality_cols_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('target', TargetEncoder())
])

In [ ]:
# bundle all the pipelines into a ColumnTransformer
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_cols_transformer, numerical_cols),
        ('ohe', ohe_cols_transformer, ohe_cols),
        ('ord', ordinal_cols_transformer, ordinal_cols),
        ('hcc', high_cardinality_cols_transformer, high_cardinality_cols)
    ]
)

In [ ]:
# using AdaBoost as the model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

model = AdaBoostClassifier(
    base_estimator = DecisionTreeClassifier(max_depth=1),
    n_estimators = 50,
    learning_rate = 0.05,
    random_state = 0
)

In [ ]:
# train the model and get the predictions
from sklearn.metrics import mean_squared_error

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

pipeline.fit(X_train, y_train),
predictions = pipeline.predict(X_valid)
score = mean_squared_error(y_valid, predictions)
print(f'Score: {score}')

In [ ]:
# get the test predictions
titanic_test = pd.read_csv('/kaggle/input/titanic/test.csv')

titanic_test['FamMembers'] = titanic_test['SibSp'] + titanic_test['Parch']
X_test = titanic_test[X_train.columns]

final_predictions = pipeline.predict(X_test)

submission = pd.DataFrame({
    'PassengerId': titanic_test['PassengerId'],
    'Survived': final_predictions
})

submission.to_csv('submission.csv', index=False)